# `S`um of `M`inimum of `C`onvex 

<b>author</b>: @guiguiom 

<b>package name</b>: SMC

<b>package type</b>: [CVXPY](https://www.cvxpy.org) extension

In [1]:
## basic imports 
import numbers
import numpy as np
import cvxpy as cp
from cvxpy.expressions.expression import Expression
from cvxpy.expressions.constants import Constant
from scipy.special import softmax
import itertools
%config InlineBackend.figure_format = 'retina'
import warnings
warnings.filterwarnings('ignore')
from IPython.display import clear_output
clear_output()

### utils

In [2]:
# Author: Mathieu Blondel
# License: BSD 3 clause


def proj_simplex_vec(V, z=1):
    n_features = V.shape[1]
    U = np.sort(V, axis=1)[:, ::-1]
    z = np.ones(len(V)) * z
    cssv = np.cumsum(U, axis=1) - z[:, np.newaxis]
    ind = np.arange(n_features) + 1
    cond = U - cssv / ind > 0
    rho = np.count_nonzero(cond, axis=1)
    theta = cssv[np.arange(len(V)), rho - 1] / rho
    return np.maximum(V - theta[:, np.newaxis], 0)

def proj_simplex(v, z=1):
    n_features = v.shape[0]
    u = np.sort(v)[::-1]
    cssv = np.cumsum(u) - z
    ind = np.arange(n_features) + 1
    cond = u - cssv / ind > 0
    rho = ind[cond][-1]
    theta = cssv[cond][-1] / float(rho)
    w = np.maximum(v - theta, 0)
    return w

### objective creation

In [3]:
### 
### 
### 
# new class
### 
### 
### 

class MinExpr:
    """
    class used to represent the minimum of cvxpy expressions 
    --------------------------------------------------------
    
    Attributes: 
    
    expr_array : list or 2d-array of CONVEX cvxpy Expression 
        LIST
        | every Expression must exhibit same dimension 
        | within each Expression, a single dtype is allowed
    dim : int
        | represents the "depth"/"dimension" of Expression objects stored in expr_array
    NOTES:
        
        ARRAY entry at (pos1,pos2) <=> LIST element at (pos2) regarding dimension pos1
        
    """
    
    def __new__(cls, *args, **kwargs):
        return super().__new__(cls)

    def __init__(self, expr_array):
        self.expressions = None
        if isinstance(expr_array,Expression): # array case
            self.expressions = expr_array.copy()
        elif isinstance(expr_array,list): # list case 
            try:
                self.expressions = cp.vstack(expr_array.copy()).T
                assert isinstance(self.expressions,Expression),'one argument passed does not match cvxpy Expression format'
            except:
                print("error while creating MinExpr object:: check dimensions of list content arguments")
        else:
            print("expr_array should be either a cvxpy Expression array or a list of Expression of equivalent sizes")

        if len(self.expressions.shape)<2:
            self.dim = 1
        elif len(self.expressions.shape)==2:
            self.dim = self.expressions.shape[0]
            if self.dim==1: # flatten() equivalent
                self.expressions = self.expressions[0]
        else: 
            print("expr_array must be a 1 or 2 dimensional")
       
        assert self.expressions.is_convex(), "expr must be convex"

    @property
    def value(self):
        """evaluates min_l=1...n of Expr_l for each pos1"""
        if self.dim==1:
            return np.min(self.expressions.value)
        else: 
            return np.min(self.expressions.value,1)
        
    @property
    def extended_values(self):
        """evaluates every component"""
        return self.expressions.value
    
    @property 
    def ns(self):
        if self.dim==1:
            return [int(np.prod(self.expressions.shape))]
        return self.dim*[int(self.expressions.shape[1])]
    
    def __add__(self, e):
        if isinstance(e, MinExpr):
            return SumMinExpr(list_min_exprs=[self,e])
        elif isinstance(e, SumMinExpr):
            return e+self
        elif isinstance(e, numbers.Number):
            self.expressions += Constant(e)
            return self
        elif isinstance(e, Expression):
            assert e.is_convex(),'e should be a cvxpy CONVEX Expression'
            try:
                self.expressions += e
            except:
                print('dimensions mismatch')
            return self
        else:
            raise ValueError("type %s not supported in __add__" % type(e))
            
    __radd__ = __add__
    
    def __mul__(self, e):
        """multiplies the MinExpr to another object
        
        Parameters
        ----------
        e : numbers.Number or np.ndarray (POSITIVE)
        
        Raises
        ------
        ValueError
            if e is not a supported type.
        Returns
        -------
        SumMinExpr
        """
        assert isinstance(e, numbers.Number) or isinstance(e,np.ndarray) or instance(e,Constant),'e should be a scalar or an np.ndarray'
        if (isinstance(e,Constant) and e.is_nonneg()):
            try:
                self.expressions = cp.multiply(e,self.expressions)
            except:
                raise ValueError('dimension not matching between e and self.expressions')
            return self
        elif np.min(e)>=0:
            try:
                self.expressions = cp.multiply(Constant(e),self.expressions)
            except:
                raise ValueError('dimension not matching between e and self.expressions')
            return self
        else:
            raise ValueError("type %s not supported in __add__" % type(e))
            
    __rmul__ = __mul__ 
    
    def clip(self,lamb):
        """takes the minimum between a MinExpr and a scalar lamb
        
        Parameters
        ----------
        lamb : numbers.Number 

        Returns
        -------
        MinExpr
        """
        assert isinstance(lamb,numbers.Number),'lamb should be a number'
        if self.dim>1:
            self.expressions = cp.hstack((self.expressions,lamb*np.ones((self.dim,1))))
        else:
            self.expressions = cp.hstack((self.expressions,lamb))
            
    def compress(self,weights=None):
        """produces the cvxpy CONVEX Expression sum_l=1...ns weight_l*Expr_l
        
        Parameters
        ----------
        weights : None (default) or np.ndarray
            | weights being positive and of length ns

        Returns
        -------
        Expression 
        """
        loc_ns = self.ns[0]
        if weights is None:
            weights = np.ones(loc_ns)/loc_ns
        if self.dim==1:
            assert len(weights)==loc_ns and (np.array(weights)>=0).all(),'weights should be a positive vector of length ns'
            sw_ = sum(weights)
            return np.array(weights)/sw_@self.expressions
        else:
            assert weights.shape == (self.dim,loc_ns) and (np.array(weights)>=0).all(),'weights should be a positive matrix of size (self.dim,ns)'
            sw_ = np.sum(weights,1)
            return cp.sum(cp.multiply(np.array(weights)/np.outer(np.ones(self.dim),sw_),self.expressions))
        
        
    def param_expand(self):
        """produces the parametric cvxpy CONVEX Expression sum_l=1...ns weight_l*Expr_l
           with newly instanciated cvxpy POSITIVE parameters weight_l for l=1...ns
        
        Returns
        -------
        Expression (parametric)
        """
        new_param = cp.Parameter(self.expressions.shape,nonneg=True)
        return cp.sum(cp.multiply(new_param,self.expressions)),[new_param]
    
### 
### 
### 
# new class
### 
### 
### 

    
class SumMinExpr:
    """
    class used to represent a sum of MinExpr
    ----------------------------------------
    
    Attributes:
    
    main_expr: Expression
        | CONVEX cvxpy Expression that stands outside Min operators (see doc.)
        
    Methods:

    __add__(e)
        -> adds the object to an Expression, MinExpr, SumOfMinExpr, or numbers.Number
    """
    
    def __new__(cls, *args, **kwargs):
        return super().__new__(cls)

    def __init__(self, list_min_exprs,main_fun=None):
        if main_fun is not None:
            assert isinstance(main_fun,Expression),'main (common) expression should be a cvxpy Expression'
            assert main_fun.is_convex(),'main (common) expression should be convex'
            self.main_expr = main_fun.copy()
        else:
            self.main_expr = Constant(0.0)
        self.min_exprs = list_min_exprs
        
    
    @property
    def num_exprs(self):
        'returns N in our formulation of SMC'
        return sum([me.dim for me in self.min_exprs])
    
    @property
    def ns_list(self):
        buf = []
        for me in self.min_exprs:
            buf.append(me.ns)
        return buf

    def __add__(self, e):
        """adds the SumMinExpr to another object
        
        Parameters
        ----------
        e : Expression, MinExpr, SumMinExpr, or numbers.Number
        Raises
        ------
        ValueError
            if e is not a supported type.
        Returns
        -------
        SumMinExpr
        """
        if isinstance(e, MinExpr):
            self.min_exprs += [e]
            return self
        elif isinstance(e, SumMinExpr):
            self.min_exprs += e.min_exprs
            self.main_expr += e.main_expr
            return self
        elif isinstance(e, numbers.Number):
            self.main_expr += Constant(e)
            return self
        elif isinstance(e, Expression):
            assert e.is_convex(),'e should be a cvxpy CONVEX Expression'
            self.main_expr += e
            return self
        else:
            raise ValueError("type %s not supported in __add__" % type(e))
            
    __radd__ = __add__
    
    
    def __mul__(self, e):
        try:
            self.main_expr *= Constant(e)
            for me in self.min_exprs:
                me *= e
        except:
            raise ValueError('dimension not matching between e and either self.main_expr or one of the expr')
        return self
            
    __rmul__ = __mul__

    
    def clip(self, lamb):
        """clips every term of a SumMinExpr to scalar value lamb
        """
        assert isinstance(lamb,numbers.Number),'lamb should be a number'
        for me in self.min_exprs:
            me.clip(lamb)
    
    @property 
    def value(self):
        """overall value"""
        return self.main_expr.value + np.sum([me.value for me in self.min_exprs])
    
    @property
    def extended_values(self):
        return [me.extended_values for me in self.min_exprs]
    
    def compress(self,weights_list):
        comp = self.main_expr
        assert len(weights_list)==self.num_exprs,'there should be as many weighting factors as MinExpr stored in SumMinExpr object'
        for me,weights in zip(self.min_exprs,weights_list):
            comp += me.compress(weights)
        return comp
    
    def param_expand(self):
        comp = self.main_expr
        new_param_list = []
        for me in self.min_exprs:
            new_term,new_param = me.param_expand()
            comp += new_term
            new_param_list += new_param
        return comp,new_param_list
    
    # TO DO .visualize(variable_values)
    
### 
### 
### 
# static method
### 
### 
### 
def minimum(e,lamb):
    assert isinstance(lamb,numbers.Number),'lamb should be a number'
    assert isinstance(e, Expression),'e should be a cvxpy Expression'
    if len(e.shape)<2:
        return SumMinExpr([MinExpr(cp.hstack((cp.reshape(e,(e.shape[0],1)),lamb*np.ones((e.shape[0],1)))))])
    else:
        return SumMinExpr([MinExpr(cp.hstack((e,lamb*np.ones((e.shape[0],1)))))])

### Problem Creation

In [4]:
class Problem:
    """
    minimizing a SumMinExpr
    --------------------------------------------
    
    Attributes:

    objective : SumMinExpr
    constraints : list
        | list of cvxpy constraints
    vars_ : list
        | cvxpy Variables (pointers) involved in the problem
    custom_param_expand : 
        | @comment
    """

    def __init__(self, objective, constraints=[],custom_param_expand=None):
        if isinstance(objective, SumMinExpr):
            self.objective = objective
        elif isinstance(objective,cp.Minimize):
            assert objective.is_dcp(),'objective should be DCP'
            self.objective = SumMinExpr(list_min_exprs=[],main_fun=objective.expr)
        elif isinstance(objective,cp.Maximize):
            assert objective.is_dcp(),'objective should be DCP'
            self.objective = SumMinExpr(list_min_exprs=[],main_fun=-objective.expr)
        else:
            raise ValueError('objective should either be a valid cvxpy DCP Objective or SumMinExpr')
        self.constraints = constraints
        for cstr in self.constraints:
            assert cstr.is_dcp(),'constraints must be CONVEX'
        self.vars_ = []
        for min_expr in self.objective.min_exprs:
            self.vars_ += min_expr.expressions.variables()
        self.vars_ += self.objective.main_expr.variables()
        for constr in self.constraints:
            self.vars_ += constr.variables()
        self.vars_ = list(set(self.vars_))
        if custom_param_expand is None:
            self.custom_set = False
            self.param_obj_fun,self.param_pointers_list = self.objective.param_expand()
        else:
            self.custom_set = True
            self.param_obj_fun,self.param_pointers_list,self.w2p = custom_param_expand[0],custom_param_expand[1],custom_param_expand[2]


    def solve(self, method="boyd", *args, **kwargs):
        """approximately solve the problem 
        
        Parameters
        ----------
        method : str
            | 'boyd','vandessel'
        args, kwargs
        """
        if method == "boyd":
            return self._solve_GSAM(*args, **kwargs)
        elif method == "vandessel":
            return self._solve_ASAAM(*args, **kwargs)
        elif method == "am":
            return self._solve_AM(*args, **kwargs)
        raise NotImplementedError(f"method {method} not supported")
        
        
    def weights_setup(self,mode='equiv'):
        weights = []
        if mode=='equiv':
            for num_s in self.objective.ns_list:
                if len(num_s)==1:
                    weights.append(np.ones(num_s[0])/num_s[0])
                else:
                    weights.append(np.outer(np.ones(len(num_s)),np.ones(num_s[0])/num_s[0]))
        elif mode=='random':
            for num_s in self.objective.ns_list:
                if len(num_s)==1:
                    base_weight = np.random.uniform(0,1,num_s[0])
                    weights.append(base_weight/sum(base_weight))
                else:
                    base_weight = np.random.uniform(0,1,(len(num_s),num_s[0]))
                    weights.append(base_weight/np.outer(np.sum(base_weight,1),np.ones(num_s[0])))   
        return weights
        
        
    def _solve_GSAM(self,stepsize=0.2, maxIters=50, verb_=False,extra_verb_=False,
                warm_start=False, warm_start_weights=None,init_weights='equiv',tol=1e-9, **kwargs):
        """ (generalized signed Alternating Minimization)
        
        Parameters
        ----------
        stepsize : double
            | stepsize
        maxIters : int
            | maximum number of iterations (default = 50)
        tol : double
            | numerical tolerance for stopping condition (default = 1e-9)
        verb_ : bool
            | whether or not to print information (default = False)
        warm_start : bool
            | whether or not some value affectation has already been conducted on Problem's variables vars_
        warm_start_weights : np.ndarray
            | choice bias; warm start value for a priori weights (default = None)
        init_weights : str or callable
            | 'random','equiv' or homemade init technique
        **kwargs
            | keyword arguments to be sent to cvxpy solve() function
        
        Returns
        -------
        info : dict
            | dictionary of solver information
        """
        if warm_start_weights is not None:
            weights = warm_start_weights.copy()
        else:
            if isinstance(init_weights,str):
                weights = self.weights_setup(mode=init_weights)
            else:
                try:
                    weights = init_weights(self)
                    # TO DO full assertive statements 
                    assert len(weights)==self.objective.num_exprs,'user prescribed weights should match objective SumMinExpr.num_exprs'
                    for elem in weights:
                        if len(elem.shape)==1:
                            assert np.min(elem)>=-1e-8 and abs(1-np.sum(elem))<=1e-8,'every weight vector should be POSITIVE and sum up to 1'
                        elif len(elem.shape)==2:
                            assert np.min(elem)>=-1e-8 and np.max(np.abs(1-np.sum(elem,1)))<=1e-8,'every weight vector should be POSITIVE and sum up to 1'
                        else:
                            raise ValueError('tensor like parameters not accepted yet...')
                except:
                    print('init_weights as a callable should take a smc.Problem argument | equiv. weights loaded...')
                    weights = self.weights_setup()
        
        cvx_param_obj_prob = cp.Problem(cp.Minimize(self.param_obj_fun), self.constraints)
        
        out_val = []
        
        last_val = np.inf
        decr = np.inf
            
        for k in range(maxIters):
            # x step, skipped if warm_start=True and k=0
            if not warm_start or k > 0:
                ## param affectation
                if self.custom_set:
                    params = self.w2p(weights)
                    for param,param_pointer in zip(params,self.param_pointers_list):
                        param_pointer.value = param
                else:
                    for weight,param_pointer in zip(weights,self.param_pointers_list):
                        param_pointer.value = weight # copy ?
                cvx_param_obj_prob.solve(**kwargs)
                prob_value = cvx_param_obj_prob.value
                decr = max(0.0,last_val-prob_value)
                if cvx_param_obj_prob.status in ['unbounded','infeasible']:
                    raise ValueError("weights-fixed problem is %s." % cvx_param_obj_prob.status)

            # weights step
            vals_list = self.objective.extended_values
            div_linf = -np.inf
            new_weights = []
            for weight,vals in zip(weights,vals_list):
                if len(vals.shape)<2:
                    minid = np.argmin(vals)
                    gbar = np.array(vals)-vals[minid]
                    sgbar = np.sign(gbar)
                    wcand = weight-stepsize*sgbar/(len(gbar)-1)
                    wcand[minid] += stepsize*(1+sgbar[minid]/(len(gbar)-1)) 
                    wcand = proj_simplex(wcand)
                elif len(vals.shape)==2:
                    minids = np.argmin(vals,1)
                    gbar = np.array(vals)-np.outer(np.min(vals,1),np.ones(vals.shape[1]))
                    sgbar = np.sign(gbar)
                    wcand = weight-stepsize*np.sign(gbar)/(len(gbar[0])-1)
                    for _,mid in enumerate(minids):
                        wcand[_,mid] += stepsize*(1+sgbar[_,mid]/(len(gbar[0])-1))
                    wcand = proj_simplex_vec(wcand)
                else:
                    print('ValueError: wrong evaluation of component functions... please check dimensions')
                div_linf = max(div_linf,np.max(np.abs(wcand-weight)))
                new_weights.append(wcand)
                
            last_val = prob_value
            true_val = self.objective.value
            out_val.append(true_val)

            if verb_:
                print("iter. %04d | Fval. %4.4e | BICval. %4.4e " % (k + 1, true_val,prob_value))
                if extra_verb_:
                    print('used weights: '+str(weights))
                    print('variables: '+str([var.value for var in self.vars_]))
                
            if decr < tol and k>3:
                if verb_:
                    print ("-> terminated (stopping condition satisfied)")
                break
            else:
                weights = new_weights.copy()
                
        if verb_ and k == maxIters-1:
            print ("-> terminated (maximum number of iterations reached)")
            
        return {'iters':k+1,'stopping_condition':decr,'objective_values':out_val}
    

    def _solve_AM(self,maxIters=50,verb_=False,extra_verb_=False,tol=1e-9,
                warm_start=False, warm_start_weights=None,init_weights='equiv', **kwargs):
        """ (Alternating Minimization)
        
        Parameters
        ----------
        stepsize : double
            | stepsize
        maxIters : int
            | maximum number of iterations (default = 50)
        tol : double
            | numerical tolerance for stopping condition (default = 1e-9)
        verb_ : bool
            | whether or not to print information (default = False)
        warm_start : bool
            | whether or not some value affectation has already been conducted on Problem's variables vars_
        warm_start_weights : np.ndarray
            | choice bias; warm start value for a priori weights (default = None)
        init_weights : str or callable
            | 'random','equiv' or homemade init technique
        **kwargs
            | keyword arguments to be sent to cvxpy solve() function
        
        Returns
        -------
        info : dict
            | dictionary of solver information
        """
        if warm_start_weights is not None:
            weights = warm_start_weights.copy()
        else:
            if isinstance(init_weights,str):
                weights = self.weights_setup(mode=init_weights)
            else:
                try:
                    weights = init_weights(self)
                    # TO DO full assertive statements 
                    assert len(weights)==self.objective.num_exprs,'user prescribed weights should match objective SumMinExpr.num_exprs'
                    for elem in weights:
                        if len(elem.shape)==1:
                            assert np.min(elem)>=-1e-8 and abs(1-np.sum(elem))<=1e-8,'every weight vector should be POSITIVE and sum up to 1'
                        elif len(elem.shape)==2:
                            assert np.min(elem)>=-1e-8 and np.max(np.abs(1-np.sum(elem,1)))<=1e-8,'every weight vector should be POSITIVE and sum up to 1'
                        else:
                            raise ValueError('tensor like parameters not accepted yet...')
                except:
                    print('init_weights as a callable should take a smc.Problem argument | equiv. weights loaded...')
                    weights = self.weights_setup()
        
        cvx_param_obj_prob = cp.Problem(cp.Minimize(self.param_obj_fun), self.constraints)
        
        out_val = []
        
        last_val = np.inf
        decr = np.inf
            
        for k in range(maxIters):
            # x step, skipped if warm_start=True and k=0
            if not warm_start or k > 0:
                ## param affectation
                if self.custom_set:
                    params = self.w2p(weights)
                    for param,param_pointer in zip(params,self.param_pointers_list):
                        param_pointer.value = param
                else:
                    for weight,param_pointer in zip(weights,self.param_pointers_list):
                        param_pointer.value = weight # copy ?
                cvx_param_obj_prob.solve(**kwargs)
                prob_value = cvx_param_obj_prob.value
                decr = max(0.0,last_val-prob_value)
                if cvx_param_obj_prob.status in ['unbounded','infeasible']:
                    raise ValueError("weights-fixed problem is %s." % cvx_param_obj_prob.status)

            # weights step
            vals_list = self.objective.extended_values
            new_weights = []
            for weight,vals in zip(weights,vals_list):
                if len(vals.shape)<2:
                    wcand = np.zeros(vals.shape)
                    wcand[np.argmin(vals)] += 1
                elif len(vals.shape)==2:
                    wcand = np.zeros(vals.shape)
                    minids = np.argmin(vals,1)
                    for _,mid in enumerate(minids):
                        wcand[_,mid] += 1
                else:
                    print('ValueError: wrong evaluation of component functions... please check dimensions')
                new_weights.append(wcand)
                    
            last_val = prob_value
            true_val = self.objective.value
            out_val.append(true_val)

            if verb_:
                print("iter. %04d | Fval. %4.4e | BICval. %4.4e | " % (k + 1, true_val,prob_value))
                if extra_verb_:
                    print('used weights: '+str(weights))
                    print('variables: '+str([var.value for var in self.vars_]))
                
            if decr < tol and k>3:
                if verb_:
                    print ("-> terminated (stopping condition satisfied)")
                break
            else:
                weights = new_weights.copy()
                
        if verb_ and k == maxIters-1:
            print ("-> terminated (maximum number of iterations reached)")
            
        return {'iters':k+1,'stopping_condition':decr,'objective_values':out_val}
    

    
    def _solve_ASAAM(self,maxIters=50, tol=1e-9, verb_=False,extra_verb_=False,warm_start_weights=None,init_weights='equiv',\
                     min_decr = 1e-5,scale_up=3/2,scale_down=2, **kwargs):
        """ (adaptive simulated annealing Alternating Minimization)
        
        Parameters
        ----------
        schedule_constant : double
            | ...
        burnin : int
            | burnin phase length;
        maxIters : int
            | maximum number of iterations (default = 50)
        tol : double
            | numerical tolerance for stopping condition (default = 1e-9)
        verb_ : bool
            | whether or not to print information (default = False)
        warm_start_weights : np.ndarray
            | choice bias; warm start value for a priori weights (default = None)
        init_weights : str or callable
            | 'random','equiv' or homemade init technique
        **kwargs
            | keyword arguments to be sent to cvxpy solve() function
        
        Returns
        -------
        info : dict
            | dictionary of solver information
        """
        if warm_start_weights is not None:
            weights = warm_start_weights.copy()
        else:
            if isinstance(init_weights,str):
                weights = self.weights_setup(mode=init_weights)
            else:
                try:
                    weights = init_weights(self)
                    # TO DO full assertive statements 
                    assert len(weights)==self.objective.num_exprs,'user prescribed weights should match objective SumMinExpr.num_exprs'
                    for elem in weights:
                        if len(elem.shape)==1:
                            assert np.min(elem)>=-1e-8 and abs(1-np.sum(elem))<=1e-8,'every weight vector should be POSITIVE and sum up to 1'
                        elif len(elem.shape)==2:
                            assert np.min(elem)>=-1e-8 and np.max(np.abs(1-np.sum(elem,1)))<=1e-8,'every weight vector should be POSITIVE and sum up to 1'
                        else:
                            raise ValueError('tensor like parameters not accepted yet...')
                except:
                    print('init_weights as a callable should take a smc.Problem argument | equiv. weights loaded...')
                    weights = self.weights_setup()
                    
        # line-serach built-up 
        temp,maxtemp,mintemp = -1.0,-1.0,-(2**15)
        
        cvx_param_obj_prob = cp.Problem(cp.Minimize(self.param_obj_fun), self.constraints)
        
        out_val = []
        
        last_val = np.inf
            
        for k in range(maxIters):
            
            decr = 0.0
            
            if k>0:
                vals_list = self.objective.extended_values
                base_cands = []
                weights = []

                # weights step -> temp>-np.inf
                for vals in vals_list:
                    if len(vals.shape)==1:
                        meanval = np.mean(vals)
                        wcand = (vals-meanval)/np.maximum(1,np.abs(meanval))
                        base_cands.append(wcand)
                        if temp>-np.inf:
                            weights.append(softmax(wcand*temp))
                        else:
                            buf = np.zeros(wcand.shape)
                            buf[np.argmin(wcand)]+=1
                            weights.append(buf)
                    elif len(vals.shape)==2:
                        meanval = np.mean(vals,1)
                        matmean = np.outer(meanval,np.ones(vals.shape[1]))
                        wcand = (vals-matmean)/np.maximum(1,np.abs(matmean))
                        base_cands.append(wcand)
                        if temp>-np.inf:
                            weights.append(softmax(wcand*temp,axis=1))
                        else:
                            buf = np.zeros(wcand.shape)
                            minids = np.argmin(wcand,1)
                            for _,mid in enumerate(minids):
                                buf[_,mid] +=1
                            weights.append(buf)
                    else:
                        print('ValueError: wrong evaluation of component functions... please check dimensions')
                        return {'iters':k+1,'stopping_condition':None,'objective_values':out_val}
                    
            
            condition = True
            while condition:
                
                # x step
                ## param affectation
                if self.custom_set:
                    params = self.w2p(weights)
                    for param,param_pointer in zip(params,self.param_pointers_list):
                        param_pointer.value = param
                else:
                    for weight,param_pointer in zip(weights,self.param_pointers_list):
                        param_pointer.value = weight # copy ?
                cvx_param_obj_prob.solve(**kwargs)
                prob_value = cvx_param_obj_prob.value
                decr = max(0.0,last_val-prob_value)
                if cvx_param_obj_prob.status in ['unbounded','infeasible']:
                    raise ValueError("weights-fixed problem is %s." % cvx_param_obj_prob.status)
                    
                    
                condition = decr<min_decr and temp>-np.inf

                # check decrease
                if decr>=min_decr:
                    printemp=temp
                    if temp==-np.inf:
                        temp = mintemp # @check
                    else:
                        temp = min(maxtemp,temp/scale_up)
                    last_val = prob_value
                elif temp>-np.inf:
                    temp = temp*scale_down
                    if temp<mintemp:
                        temp = -np.inf
                    if extra_verb_:
                        print('backtracking...freezing more | temp = '+str(temp))
                    weights = []
                    for wcand in base_cands:
                        if len(wcand.shape)==1:
                            if temp>-np.inf:
                                weights.append(softmax(wcand*temp))
                            else:                    
                                buf = np.zeros(wcand.shape)
                                buf[np.argmin(wcand)] += 1
                                weights.append(buf)
                        elif len(wcand.shape)==2:
                            if temp>-np.inf:
                                weights.append(softmax(wcand*temp,axis=1))
                            else:
                                buf = np.zeros(wcand.shape)
                                minids = np.argmin(wcand,1)
                                for _,mid in enumerate(minids):
                                    buf[_,mid] += 1
                                weights.append(buf)
                        else:
                            print('ValueError: wrong evaluation of component functions... please check dimensions')
                            return {'iters':k+1,'stopping_condition':decr,'objective_values':out_val}
                    
            true_val = self.objective.value
            out_val.append(true_val)

            if verb_:
                print("iter. %04d | Fval. %4.4e | BICval. %4.4e | decr. %4.4e | temp. %4.4e" % (k + 1, true_val,prob_value, decr,printemp))
                if extra_verb_:
                    print('used weights: '+str(weights))
                    print('variables: '+str([var.value for var in self.vars_]))
                
            if (decr < tol and k>3) or temp<=-np.inf:
                if verb_:
                    print ("-> terminated (stopping condition satisfied)")
                break
                
        if verb_ and k == maxIters-1:
            print ("-> terminated (maximum number of iterations reached)")
            
        return {'iters':k+1,'stopping_condition':decr,'objective_values':out_val}
    
    
    @property 
    def value(self):
        for cstr in self.constraints:
            if cstr.value()==False:
                return np.inf # infeasibility
        return self.objective.value

### Specific WARM-STARTS